Vamos a realizar una tarea de clasificacion con una CNN completa, usando MNIST como dataset, el "hellow world" de CV

### Carga del Dataset

In [85]:
# Arranque: imports, dataset y primer batch (MNIST)
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


torch.manual_seed(3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Aqui convertimos las imagenes a tensores
# Es decir, el valor de los pixeles pasa a estar entre 0 y 1
# y el shape de las imagenes pasa a ser (C,H,W) (Canales, Alto, Ancho)

transform = transforms.ToTensor()  # [0,1], shape (C,H,W)

train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

loader_train = DataLoader(train, batch_size=128, shuffle=True)
loader_test = DataLoader(test, batch_size=128, shuffle=True)


images, labels = next(iter(loader_train))
print(images.shape, labels.shape)





cuda
torch.Size([128, 1, 28, 28]) torch.Size([128])


In [86]:
#Comprobamos que el shape de las imagenes es el esperado
assert images.ndim == 4
assert images.shape[1] == 1
assert images.shape[2:] == (28, 28)
assert images.min() >= 0.0 and images.max() <= 1.

#### Definición de arquitectura CNN


In [87]:
import torch
import torch.nn as nn

def get_flatten_size(model_features, input_shape=(1, 1, 28, 28)):
    with torch.no_grad():
        x = torch.zeros(input_shape)
        out = model_features(x)
        return out.view(out.size(0), -1).size(1)

class MiniCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.features = nn.Sequential(

            #Primera capa de convolucion, entra una imagen en blanco y negro (1 canal) y sale una imagen de 32 features (filtros)
            #El kernel es de 3x3 y el padding es 1 (same padding)
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2), #Aplicamos pooling max para reducir la dimensionalidad de la imagen

            #Segunda capa de convolucion, entra una imagen de 32 features y sale una imagen de 64 features
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            nn.MaxPool2d(kernel_size=2, stride=2) #Aplicamos pooling max para reducir la dimensionalidad de la imagen
        )

        n_flat = get_flatten_size(self.features)

        

        #Añadimos una capa lineal para clasificar
        self.classifier = nn.Sequential(
            nn.Flatten(), #Aplanamos la imagen para que sea un vector
            nn.Dropout(p=0.3), #Aplicamos dropout para evitar el overfitting
            
            nn.Linear(in_features=n_flat, out_features=64), #Una capa lineal con n_flat entradas y 64 salidas
            nn.ReLU(), #Aplicamos la funcion de activacion ReLU
            
            nn.Dropout(p=0.5), #Aplicamos dropout para evitar el overfitting
            
            nn.Linear(in_features=64, out_features=10), #Una capa lineal con 64 entradas y 10 salidas
            # No aplicamos softmax, ya que la funcion de perdida que usamos (CrossEntropyLoss) lo aplica por defecto
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x





# Instancia
model = MiniCNN()
print(model)

x = torch.randn(4, 1, 28, 28)
logits = model(x)
print(logits.shape)  # esperado: (4, 10)


MiniCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dropout(p=0.3, inplace=False)
    (2): Linear(in_features=3136, out_features=64, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=64, out_features=10, bias=True)
  )
)
torch.Size([4, 10])


Vamos a imprimir un summary

PyTorch usa formato NCHW:

| Posición | Significado                         | Ejemplo |
| -------- | ----------------------------------- | ------- |
| 0        | N → número de imágenes (batch size) | 4   (-1 = None (no definido)    |
| 1        | C → canales o *features maps* (filtros)       | 32      |
| 2        | H → alto de la imagen               | 28      |
| 3        | W → ancho de la imagen              | 28      |


In [88]:
from torchsummary import summary

summary(model.to(device), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
           Dropout-8                 [-1, 3136]               0
            Linear-9                   [-1, 64]         200,768
             ReLU-10                   [-1, 64]               0
          Dropout-11                   [-1, 64]               0
           Linear-12                   [-1, 10]             650
Total params: 220,234
Trainable params: 220,234
Non-trainable params: 0
-------------------------------

### Vamos a declarar el bucle de entramiento

Vamos a ver los parametros tipicos de SDG

| Parámetro  | Rol                                                      | Valor típico |
| ---------- | -------------------------------------------------------- | ------------ |
| `lr`       | tamaño del paso (cuánto cambian los pesos por gradiente) | 0.01–0.1     |
| `momentum` | cuánto “recuerda” del gradiente anterior                 | 0.8–0.95     |


Declaramos funcion de error y optimizador (gradientre)

In [89]:
EPOCHS = 9  # número de pasadas por el dataset
lr = 0.01
momentum = 0.9
#Definimos la funcion de perdida y el optimizador
criterion = nn.CrossEntropyLoss()

#lr es la tasa de aprendizaje, momentum es el factor de inercia, es decir, cuanto se mueve el optimizador en la direccion del gradiente y 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)



Declaramos los epochs y activamos el modelo.

In [90]:

model.to(device)  # mueve el modelo a GPU
model.train()     # pone el modelo en modo entrenamiento (activa dropout, etc.)


MiniCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dropout(p=0.3, inplace=False)
    (2): Linear(in_features=3136, out_features=64, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [91]:
for epoch in range(1, EPOCHS+1):
    total, correct = 0, 0
    running_loss = 0.0
    print(f"[inicio] epoch {epoch}")


[inicio] epoch 1
[inicio] epoch 2
[inicio] epoch 3
[inicio] epoch 4
[inicio] epoch 5
[inicio] epoch 6
[inicio] epoch 7
[inicio] epoch 8
[inicio] epoch 9


In [92]:
def evaluate(model, loader, device, criterion):
    model.eval()
    total, correct, total_loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            total_loss += loss.item() * images.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / total, correct / total

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in loader_train:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    test_loss, test_acc = evaluate(model, loader_test, device, criterion)

    print(
        f"Época {epoch}/{EPOCHS} | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
        f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}"
    )


Época 1/9 | Train Loss: 0.6698 | Train Acc: 0.7795 | Test Loss: 0.1300 | Test Acc: 0.9591
Época 2/9 | Train Loss: 0.2182 | Train Acc: 0.9326 | Test Loss: 0.0766 | Test Acc: 0.9745
Época 3/9 | Train Loss: 0.1639 | Train Acc: 0.9508 | Test Loss: 0.0634 | Test Acc: 0.9799
Época 4/9 | Train Loss: 0.1383 | Train Acc: 0.9586 | Test Loss: 0.0518 | Test Acc: 0.9827
Época 5/9 | Train Loss: 0.1225 | Train Acc: 0.9636 | Test Loss: 0.0483 | Test Acc: 0.9845
Época 6/9 | Train Loss: 0.1079 | Train Acc: 0.9678 | Test Loss: 0.0427 | Test Acc: 0.9863
Época 7/9 | Train Loss: 0.1002 | Train Acc: 0.9705 | Test Loss: 0.0413 | Test Acc: 0.9873
Época 8/9 | Train Loss: 0.0948 | Train Acc: 0.9720 | Test Loss: 0.0373 | Test Acc: 0.9880
Época 9/9 | Train Loss: 0.0902 | Train Acc: 0.9737 | Test Loss: 0.0330 | Test Acc: 0.9887


Si train acc sube pero test acc baja entre épocas consecutivas, estás viendo overfitting; detén ahí o sube temporalmente el dropout.

En este experimento hemos llegado a un Acc de 0.98 (98%) en el train set con 9 EPOCHS.

Subir una EPOCH más (10) ya generaba Overfitting.

#### Vamos a guardar nuestro modelo.

In [94]:
#Esto guardara los pesos del modelo, no el modelo completo.

torch.save(model, "mnist_model/mnist_full.pth")

#Para guardar el modelo completo y compilado:

# Asegúrate de que el modelo esté en modo eval
model.eval()

# Convierte el modelo a TorchScript (usa script mejor que trace si hay condicionales o lógica)
scripted_model = torch.jit.script(model)

# Guarda el modelo compilado
scripted_model.save("mnist_model/minicnn_mnist_best.pt")

print("Modelo guardado como TorchScript (.pt)")



Modelo guardado como TorchScript (.pt)
